### Extending Numba to Support a Custom Particle Class

In this notebook, we'll extend Numba to support a custom Particle class. This involves allowing:

- Passing an instance of the class to a Numba function.
- Accessing attributes of the class in a Numba function.
- Constructing and returning a new instance of the class from a Numba function.

We'll accomplish this by mixing high-level and low-level extension APIs provided by Numba.

https://numba.readthedocs.io/en/stable/extending/index.html

**Step 1: Defining the Custom Particle Class**

First, we define a simple Python class Particle, which represents a charged particle in an electric field. The class has two attributes: q (charge) and E (electric field). We also define a property force, which calculates the force on the particle using the formula F = q.E

In [4]:
import numpy as np

class Particle(object):
    """
    A charged particle in an electric field.
    """
    def __init__(self, q, E):
        self.q = q
        self.E = E

    def __repr__(self):
        return 'Particle(%f, %f)' % (self.q, self.E)

    @property
    def force(self):
        return np.dot(self.E, self.q)

**Step 2: Creating a Numba Type for the Particle Class**

Numba uses its own type system, so we need to create a new Numba type for the Particle class. This is done by subclassing `types.Type` and creating an instance of this type.

In [5]:
from numba import types

class ParticleType(types.Type):
    def __init__(self):
        super(ParticleType, self).__init__(name='Particle')

particle_type = ParticleType()


Here, `ParticleType` is a custom type that Numba will recognize as representing instances of the Particle class.

**Step 3: Type Inference for Python Values**

Next, we teach Numba how to infer that instances of Particle should be treated as our custom particle_type.

In [6]:
from numba.extending import typeof_impl

@typeof_impl.register(Particle)
def typeof_index(val, c):
    return particle_type

This allows Numba to recognize function arguments and global values as particle_type when they are instances of Particle.

**Step 4: Type Inference for the Constructor**

We also need to teach Numba to recognize the constructor for the Particle class, specifically the Particle(q, E) constructor where q and E are floating-point numbers.

In [7]:
from numba.extending import type_callable

@type_callable(Particle)
def type_particle(context):
    def typer(q, E):
        if isinstance(q, types.Float) and isinstance(E, types.Float):
            return particle_type
    return typer

The type_callable decorator allows us to specify a function that will be called during type inference for the Particle constructor.

**Step 5: Defining the Data Model for Particle**

In nopython mode, Numba uses native data representations instead of Python objects. We define a data model for the Particle class using a structure with fields q and E, both of type float64.

In [8]:
from numba.extending import models, register_model

@register_model(ParticleType)
class ParticleModel(models.StructModel):
    def __init__(self, dmm, fe_type):
        members = [
            ('q', types.float64),
            ('E', types.float64),
            ]
        models.StructModel.__init__(self, dmm, fe_type, members)

This structure allows Numba to store instances of Particle as a simple struct with two double-precision floating-point fields.

**Step 6: Exposing Attributes to Numba Functions**

We expose the q and E attributes of the Particle class to Numba functions using make_attribute_wrapper.

In [9]:
from numba.extending import make_attribute_wrapper

make_attribute_wrapper(ParticleType, 'q', 'q')
make_attribute_wrapper(ParticleType, 'E', 'E')

This allows these attributes to be accessed in read-only mode within Numba functions.

**Step 7: Exposing the Force Property**

The force property is computed rather than stored, so we need to implement it explicitly for use in Numba functions.

In [10]:
from numba.extending import overload_attribute

@overload_attribute(ParticleType, "force")
def get_force(particle):
    def getter(particle):
        return particle.E * particle.q
    return getter

The overload_attribute decorator is a high-level API that combines type inference and code generation.

**Step 8: Implementing the Particle Constructor**

We implement the two-argument Particle constructor for use in Numba functions.

In [11]:
from numba.extending import lower_builtin
from numba.core import cgutils

@lower_builtin(Particle, types.Float, types.Float)
def impl_particle(context, builder, sig, args):
    typ = sig.return_type
    q, E = args
    particle = cgutils.create_struct_proxy(typ)(context, builder)
    particle.q = q
    particle.E = E
    return particle._getvalue()

This code generates the LLVM code needed to create a Particle object from its components in a Numba function.

**Step 9: Unboxing and Boxing**

Unboxing is the process of converting a Python object to a native Numba value, while boxing does the reverse. We define these operations to allow Numba to convert between Python Particle objects and native Particle structures.

In [12]:
from numba.extending import unbox, NativeValue

@unbox(ParticleType)
def unbox_particle(typ, obj, c):
    """
    Convert a Particle object to a native particle structure.
    """
    q_obj = c.pyapi.object_getattr_string(obj, "q")
    E_obj = c.pyapi.object_getattr_string(obj, "E")
    particle = cgutils.create_struct_proxy(typ)(c.context, c.builder)
    particle.q = c.pyapi.float_as_double(q_obj)
    particle.E = c.pyapi.float_as_double(E_obj)
    c.pyapi.decref(q_obj)
    c.pyapi.decref(E_obj)
    is_error = cgutils.is_not_null(c.builder, c.pyapi.err_occurred())
    return NativeValue(particle._getvalue(), is_error=is_error)

In [13]:
from numba.extending import box

@box(ParticleType)
def box_particle(typ, val, c):
    """
    Convert a native particle structure to an Particle object.
    """
    particle = cgutils.create_struct_proxy(typ)(c.context, c.builder, value=val)
    q_obj = c.pyapi.float_from_double(particle.q)
    E_obj = c.pyapi.float_from_double(particle.E)
    class_obj = c.pyapi.unserialize(c.pyapi.serialize_object(Particle))
    res = c.pyapi.call_function_objargs(class_obj, (q_obj, E_obj))
    c.pyapi.decref(q_obj)
    c.pyapi.decref(E_obj)
    c.pyapi.decref(class_obj)
    return res

**Step 10: Using the Particle Class in Numba Functions**

Finally, we can now use the Particle class in Numba-compiled functions. Below are examples of such functions:
Example: Calculating Force

Example: Calculating Force

Example: Summing Two Particles

We can test our implementation by creating Particle objects and using them in Numba functions: